In [1]:
import os
import sys
import json
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Importing local modules from src folder
src_dir = os.path.join(os.getcwd(), '..', 'src')
if src_dir not in sys.path:
    sys.path.append(src_dir)

from model_utils import build_simple_RNN_model

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aaron\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aaron\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


ImportError: cannot import name 'build_LSTM_model' from 'model_utils' (c:\Users\aaron\Documents\GitHub\doomer-check\notebooks\..\src\model_utils.py)

In [ ]:
# Data loading
csv_file_path = '../data/processed/preprocessed_tweets.csv'
df = pd.read_csv(csv_file_path)

In [ ]:
X = df['tweet']  # Features: tweet texts
y = df['sentiment']  # Labels: sentiments

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Data preparation for NN models

In [ ]:
# Tokenization of text data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [ ]:
# Padding sequences to be of equal length
max_length = max([len(seq) for seq in X_train_seq]) # max length won't be too large since tweets are char-limited

X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

In [ ]:
# Saving tokenizer to json
json_file_path = '../data/processed/tokenizer.json'
tokenizer_json = tokenizer.to_json()
with open(json_file_path, "w", encoding="utf-8") as f:
    f.write(tokenizer_json)

### Building, training, and evaluating neural network models

In [ ]:
# Building models
rnn_model = build_simple_RNN_model(input_length=max_length, learning_rate=0.0005)

In [ ]:
# Train the RNN model
history_rnn = rnn_model.fit(X_train_padded, y_train, epochs=5, validation_split=0.2, batch_size=32)

In [ ]:
# Evaluate the RNN model
rnn_loss, rnn_acc = rnn_model.evaluate(X_test_padded, y_test)
print(f'RNN Model Accuracy: {rnn_acc}')